In [1]:
import sys
sys.path.append("..")

from app.analyzer import analyze_entry
from app.storage import save_entry, load_entries

In [3]:
journal_text = """
I’ve been feeling overwhelmed by the pace of work lately. Even when I finish something, the to-do list just keeps growing. 
I know I should be proud of what I accomplish, but I can't help feeling like I'm falling short.
"""


In [5]:
result = analyze_entry(journal_text)
save_entry(journal_text, result)


✅ Entry saved at 2025-06-09T20:35:22.553640
🧠 Appended to RAG file: /Users/Additional Storage/ML Projects/Journal app/notebooks/../app/../data/rag/rag_journal_entries.jsonl


In [6]:
entries = load_entries()
entries[-1]


{'text': "\nI’ve been feeling overwhelmed by the pace of work lately. Even when I finish something, the to-do list just keeps growing. \nI know I should be proud of what I accomplish, but I can't help feeling like I'm falling short.\n",
 'analysis': {'summary': 'Feeling overwhelmed by the pace of work despite completing tasks.',
  'emotions': ['overwhelmed', 'frustrated'],
  'patterns': ['catastrophizing', 'self-blame', 'hopelessness'],
  'themes': ['work stress', 'personal achievements', 'work balance'],
  'ai_thoughts': "Okay, so I need to analyze this journal entry and provide a summary, primary emotions, cognitive distortions, and core themes using JSON format. Let me break it down step by step.\n\nFirst, the user is talking about being overwhelmed by work pace. They finish tasks but the list keeps growing. That's creating stress or anxiety because there's always more to do even after completing something. So the summary should capture that feeling of overwhelm and the ongoing task

In [8]:
from app.storage import load_entries

entries = load_entries()

# Find the entry by keyword (e.g., 'new Journal')
for i, e in enumerate(reversed(entries)):
    if "new Journal" in e["text"]:
        print(f"Found at index: {-1 - i}")
        print(e["text"])
        break


In [9]:
entry = entries[-2]
print("📝 Text:\n", entry["text"])
print("🧠 Summary:\n", entry["summary"])
print("🧾 Raw Output:\n", entry.get("raw_output", "No raw output found"))


📝 Text:
 I am checking if the new RAG works, I hope it does and le'ts see - I am nervous about it 123456789 practicing counting numbers let's try something new may be that will workkk umm lets see RAG
NLP


KeyError: 'summary'

In [ ]:
from app.analyzer import analyze_entry

In [ ]:
result = analyze_entry(entry['text'])


In [ ]:
result

In [ ]:
# generate_realistic_journals.py
# Uses DeepSeek (via Ollama) to create 150 diverse journal entries

import json
import random
import time
from datetime import datetime, timedelta
import requests

# Constants
OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME = "deepseek-r1"
OUTPUT_PATH = "/Users/Additional Storage/ML Projects/Journal app/data/generated_deepseek_entries.jsonl"

# Emotion/theme seeds for diversity
emotions_pool = [
    "anxiety", "joy", "loneliness", "hope", "confusion", "pride", "grief", "boredom", "motivation",
    "excitement", "fear", "regret", "curiosity", "burnout", "peace"
]
topics_pool = [
    "relationships", "work stress", "family dynamics", "career growth", "mental health", "self-discovery",
    "routine fatigue", "health journey", "romantic feelings", "loss", "uncertainty about future",
    "celebrating a win", "dealing with rejection", "spiritual questioning"
]

# Prompt to generate realistic journal text
ENTRY_GENERATION_PROMPT = """
You're a human writing a private journal.
Your tone should be raw, informal, and personal.
Write a journal entry about feeling {emotion} today, with thoughts related to {topic}.
Include emotion, thought loops, and sensory detail if helpful.
Keep it between 4–8 sentences.
Do not include headings or lists.
Just return the journal text.
"""

import re

# Generate a single journal text from deepseek
def generate_journal_text(emotion, topic):
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": "You are a human journaling assistant."},
            {"role": "user", "content": ENTRY_GENERATION_PROMPT.format(emotion=emotion, topic=topic)}
        ],
        "stream": False
    }
    try:
        response = requests.post(OLLAMA_URL, json=payload)
        raw = response.json()["message"]["content"].strip()
        cleaned = re.sub(r"<think>.*?</think>", "", raw, flags=re.DOTALL).strip()
        return cleaned
    except Exception as e:
        print(f"❌ Failed to generate entry for {emotion} + {topic}: {e}")
        return None

# Generate realistic timestamp
def random_timestamp(start_year=2024, end_year=2025):
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    total_days = (end_date - start_date).days
    random_days = random.randint(0, total_days)
    random_seconds = random.randint(0, 86400)
    return (start_date + timedelta(days=random_days, seconds=random_seconds)).isoformat()

# Generate entries and save
entries = []
combos = list(set((e, t) for e in emotions_pool for t in topics_pool))
random.shuffle(combos)

total = 2
print(f"🧠 Generating {total} entries via DeepSeek…")

for i in range(total):
    emotion, topic = combos[i % len(combos)]
    text = generate_journal_text(emotion, topic)
    if not text:
        continue

    entry = {
        "timestamp": random_timestamp(),
        "text": text,
        "summary": "",  # To be filled by your analyzer
        "emotions": [],
        "patterns": [],
        "themes": [],
        "ai_thoughts": "",
        "raw_output": ""
    }
    entries.append(entry)
    print(f"✅ {i+1}/{total}: {emotion}, {topic}")
    time.sleep(1.5)  # Optional pacing if DeepSeek needs time

# Save to .jsonl
with open(OUTPUT_PATH, "w") as f:
    for entry in entries:
        f.write(json.dumps(entry) + "\n")

print(f"\n✅ Done. Saved {len(entries)} entries to: {OUTPUT_PATH}")


In [ ]:
entries = []

with open(OUTPUT_PATH, "r") as f:
    for line in f:
        entries.append(json.loads(line.strip()))

# Now you can access individual ones like:
print(entries[0]["text"])


In [ ]:
entries = []

with open(OUTPUT_PATH, "r") as f:
    for line in f:
        entries.append(json.loads(line.strip()))

# Now you can access individual ones like:
print(entries[0])


In [ ]:
entries = []

with open(OUTPUT_PATH, "r") as f:
    for line in f:
        entries.append(json.loads(line.strip()))

# Now you can access individual ones like:
print(entries[5]['text'])


In [ ]:
# generate_realistic_journals.py
import json, random, re, requests
from datetime import datetime, timedelta
from pathlib import Path

OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME = "deepseek-r1"
OUTPUT_PATH = "/Users/Additional Storage/ML Projects/Journal app/data/test_generated_entries.jsonl"
TOTAL_ENTRIES = 100
TEMPERATURE = 0.8

emotions = ["anxiety", "joy", "confusion", "hope", "frustration", "nostalgia", "burnout", "contentment", "loneliness"]
topics = ["work stress", "relationships", "self-doubt", "mental health", "growth", "grief", "career uncertainty", "romantic confusion", "burnout recovery", "loneliness", "change and transition", "creative block", "financial pressure", "family expectations", "isolation", "identity", "body image", "success anxiety", "fear of failure", "disconnection from others", "social anxiety"]

GEN_PROMPT = """
You are a private journal writer.
Write a thoughtful, emotionally realistic journal entry from someone who is experiencing the emotion: {emotion}, in the context of: {topic}.
The tone should feel personal and introspective — not scripted / cmputer written, it should sound like a normal decently educated indian english writer.
Write 4–8 flowing sentences, including sensory details and authentic reflection.
Ensure good grammar and spelling, and avoid artificial phrasing or markdown.
Return only the journal entry text with no commentary or extra formatting.
"""

def generate_entry(emotion, topic):
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": "You are a human journaling assistant."},
            {"role": "user", "content": GEN_PROMPT.format(emotion=emotion, topic=topic)}
        ],
        "temperature": TEMPERATURE,
        "stream": False
    }
    try:
        res = requests.post(OLLAMA_URL, json=payload)
        raw = res.json()["message"]["content"].strip()
        return re.sub(r"<think>.*?</think>", "", raw, flags=re.DOTALL).strip()
    except Exception as e:
        print(f"❌ Error for {emotion}/{topic}: {e}")
        return None

def analyze_entry(text):
    ANALYSIS_PROMPT = """
You are a compassionate AI assistant trained to analyze journal entries.
Return ONLY a valid raw JSON object with the following keys:
  - summary: a one-sentence paraphrase of the overall entry
  - emotions: list of 1–3 dominant emotional states
  - patterns: list of up to 3 cognitive distortions (e.g., catastrophizing)
  - themes: list of up to 3 core life topics the entry relates to

Do NOT include explanations or markdown.
"""
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": ANALYSIS_PROMPT},
            {"role": "user", "content": text}
        ],
        "stream": False
    }
    try:
        res = requests.post(OLLAMA_URL, json=payload)
        content = res.json()["message"]["content"].strip()
        match = re.search(r"{.*}", content, re.DOTALL)
        return json.loads(match.group(0)) if match else None
    except Exception as e:
        print(f"❌ Analysis error: {e}")
        return None

def random_timestamp():
    start = datetime(2024, 1, 1)
    end = datetime(2024, 12, 31)
    delta = end - start
    offset = timedelta(days=random.randint(0, delta.days), hours=random.randint(18, 22), minutes=random.randint(0, 59))
    return (start + offset).isoformat()

def emotion_for_month(month):
    bias_map = {
        1: ["anxiety", "burnout"], 2: ["anxiety"], 3: ["confusion"], 4: ["hope"], 5: ["burnout"],
        6: ["joy", "contentment"], 7: ["joy"], 8: ["nostalgia"], 9: ["confusion", "anxiety"],
        10: ["nostalgia", "loneliness"], 11: ["loneliness"], 12: ["loneliness", "contentment"]
    }
    biased = bias_map.get(month, [])
    return random.choice(biased) if biased and random.random() < 0.6 else random.choice(emotions)

# Load existing
existing = []
if Path(OUTPUT_PATH).exists():
    with open(OUTPUT_PATH) as f:
        existing = [json.loads(line) for line in f if line.strip()]
existing_ts = set(e["timestamp"] for e in existing)

# Generate
new_entries = []
for _ in range(TOTAL_ENTRIES):
    ts = random_timestamp()
    if ts in existing_ts:
        continue
    emo = emotion_for_month(datetime.fromisoformat(ts).month)
    top = random.choice(topics)
    text = generate_entry(emo, top)
    if not text:
        continue
    analysis = analyze_entry(text)
    if not analysis:
        continue
    new_entries.append({
        "timestamp": ts,
        "text": text,
        "analysis": analysis
    })
    print(f"✅ {ts} — {emo} | {top}")

# Save all
with open(OUTPUT_PATH, "w") as f:
    for e in existing + new_entries:
        f.write(json.dumps(e) + "\n")

print(f"\n📄 Saved {len(new_entries)} new entries to: {OUTPUT_PATH}")


In [ ]:
import os
import json

DATA_DIR = "/Users/Additional Storage/ML Projects/Journal app/data"

for filename in os.listdir(DATA_DIR):
    if filename.endswith(".jsonl") and not filename.startswith("rag_"):
        input_path = os.path.join(DATA_DIR, filename)
        output_path = os.path.join(DATA_DIR, f"rag_{filename}")

        rag_entries = []
        with open(input_path, "r") as f:
            for line in f:
                try:
                    entry = json.loads(line.strip())
                    rag_doc = {
                        "content": entry.get("text", ""),
                        "metadata": {
                            "timestamp": entry.get("timestamp"),
                            "themes": entry.get("analysis", {}).get("themes", []),
                            "emotions": entry.get("analysis", {}).get("emotions", [])
                        }
                    }
                    rag_entries.append(rag_doc)
                except json.JSONDecodeError:
                    print(f"❌ Skipped malformed line in {filename}")

        with open(output_path, "w") as f:
            for doc in rag_entries:
                f.write(json.dumps(doc) + "\n")

        print(f"✅ Created {output_path} with {len(rag_entries)} entries.")

In [ ]:
import os

data_dir = "/Users/Additional Storage/ML Projects/Journal app/data"
jsonl_files = [f for f in os.listdir(data_dir) if f.endswith(".jsonl") and not f.startswith("rag_")]
print(jsonl_files)

In [ ]:
import json

for file in jsonl_files:
    print(f"\n📂 Checking: {file}")
    path = os.path.join(data_dir, file)
    with open(path) as f:
        for i, line in enumerate(f):
            try:
                obj = json.loads(line.strip())
                if "timestamp" not in obj:
                    print(f"❌ Missing timestamp at line {i+1}: {obj}")
            except json.JSONDecodeError as e:
                print(f"⚠️ Malformed JSON at line {i+1}: {e}")

In [11]:
clean_entries = []
bad_entries = []

with open(os.path.join(data_dir, "journal_entries.jsonl")) as f:
    for i, line in enumerate(f):
        try:
            obj = json.loads(line.strip())
            if "timestamp" in obj:
                clean_entries.append(obj)
            else:
                bad_entries.append((i + 1, obj))
        except json.JSONDecodeError as e:
            bad_entries.append((i + 1, f"Malformed JSON: {e}"))

print(f"✅ {len(clean_entries)} clean entries")
print(f"❌ {len(bad_entries)} bad entries")

✅ 1 clean entries
❌ 0 bad entries


In [ ]:
for file in jsonl_files:
    print(f"\n📁 Checking: {file}")
    path = os.path.join(data_dir, file)
    with open(path) as f:
        for i, line in enumerate(f):
            try:
                obj = json.loads(line.strip())
                if "timestamp" not in obj:
                    print(f"❌ {file} line {i+1} is missing timestamp:\n{obj}")
            except Exception as e:
                print(f"⚠️ Error in {file} line {i+1}: {e}")

In [15]:
entries = [json.loads(line.strip()) for line in open(path)]
entries.sort(key=lambda x: x["timestamp"], reverse=True)

In [40]:
import json

rag_path = '/Users/Additional Storage/ML Projects/Journal app/data/rag/rag_journal_entries.jsonl'

with open(rag_path, "r") as f:
    lines = [json.loads(line) for line in f]

print(f"Total entries: {len(lines)}")
print("Last entry preview:", lines)

Total entries: 2
Last entry preview: [{'content': "I am checking if the new RAG works, I hope it does and le'ts see - I am nervous about it 123456789 practicing counting numbers", 'metadata': {'timestamp': '2025-06-04T22:36:40.885776', 'themes': ['anxiety about technology', 'practicing a routine task', 'new tool anxiety'], 'emotions': ['anxiety', 'nervousness']}}, {'content': "I am checking if the new RAG works, I hope it does and le'ts see - I am nervous about it 123456789 practicing counting numbers let's try something new may be that will workkk umm lets see RAG\nNLP", 'metadata': {'timestamp': '2025-06-04T22:38:19.492991', 'themes': ['testing RAG model', 'trying something new', 'experimentation with AI'], 'emotions': ['nervousness', 'uncertainty', 'excitement']}}]


In [18]:
import json
import os
def load_journal_entries(file_path):
    entries = []
    with open(file_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                entries.append(json.loads(line))
            except json.JSONDecodeError:
                print(f"⚠️ Could not parse line: {line}")
    return entries

# Load your journal data
BASE_PATH = "/Users/Additional Storage/ML Projects/Journal app"
DATA_FOLDER = os.path.join(BASE_PATH, "data")
ENTRY_FILE = os.path.join(DATA_FOLDER, "test_generated_entries.jsonl")

entries = load_journal_entries(ENTRY_FILE)

print(f"✅ Loaded {len(entries)} journal entries.")
print("📝 First entry:")
print(entries[0])

✅ Loaded 119 journal entries.
📝 First entry:
{'timestamp': '2024-01-05T18:37:00', 'text': 'The room was dim, the hum of an fluorescent bulb barely audible among the heavy silence. I sat in my corner, sipping tea from a teacup so small it felt like a personal pronouncement against my cup size. The weight of tomorrow’s presentation gnawed at me, its preparation having worn a toll not just on my body but on my soul. The tension stretched taught across my shoulders, each muscle a knot where nerves had knotted them before, but today, I felt it was different. The heart thrummed faster than usual, the pounding met with a faint flutter, as if it were racing against an invisible clock.\n\nMy fingers trembled involuntarily on the edge of my laptop, fingers that had once clapped together so neatly during revisions at work. The keys beneath them felt cold to the touch now, the same cold that made my palms turn prune-like. I tried to quiet the noise inside me, to compartmentalize the fear and dread

In [20]:
from langchain.schema import Document

def convert_entries_to_documents(entries):
    documents = []

    for entry in entries:
        text = entry.get("text", "")
        analysis = entry.get("analysis", {})
        metadata = {
            "timestamp": entry.get("timestamp", ""),
            "summary": analysis.get("summary", ""),
            "themes": analysis.get("themes", []),
            "emotions": analysis.get("emotions", [])
        }
        documents.append(Document(page_content=text, metadata=metadata))

    return documents

# Convert loaded entries into LangChain documents
documents = convert_entries_to_documents(entries)

print(f"✅ Converted {len(documents)} entries into LangChain Documents")
print("🔍 Example Document:")
print(documents[0])

✅ Converted 119 entries into LangChain Documents
🔍 Example Document:
page_content='The room was dim, the hum of an fluorescent bulb barely audible among the heavy silence. I sat in my corner, sipping tea from a teacup so small it felt like a personal pronouncement against my cup size. The weight of tomorrow’s presentation gnawed at me, its preparation having worn a toll not just on my body but on my soul. The tension stretched taught across my shoulders, each muscle a knot where nerves had knotted them before, but today, I felt it was different. The heart thrummed faster than usual, the pounding met with a faint flutter, as if it were racing against an invisible clock.

My fingers trembled involuntarily on the edge of my laptop, fingers that had once clapped together so neatly during revisions at work. The keys beneath them felt cold to the touch now, the same cold that made my palms turn prune-like. I tried to quiet the noise inside me, to compartmentalize the fear and dread that pres

In [22]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
import os

# Step 1: Define paths
BASE_PATH = "/Users/Additional Storage/ML Projects/Journal app"
DATA_FILE = os.path.join(BASE_PATH, "data", "test_generated_entries.jsonl")

# Step 2: Load and convert entries to LangChain Documents
from langchain.schema import Document
import json

def load_and_convert_to_documents(file_path):
    documents = []
    with open(file_path, "r") as f:
        for line in f:
            if not line.strip():
                continue
            try:
                entry = json.loads(line)
                text = entry.get("text", "")
                analysis = entry.get("analysis", {})
                metadata = {
                    "timestamp": entry.get("timestamp", ""),
                    "summary": analysis.get("summary", ""),
                    "themes": analysis.get("themes", []),
                    "emotions": analysis.get("emotions", [])
                }
                documents.append(Document(page_content=text, metadata=metadata))
            except json.JSONDecodeError:
                continue
    return documents

documents = load_and_convert_to_documents(DATA_FILE)

# Step 3: Embed and create vectorstore
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents, embedding_model)

# ✅ Step 4: Save the index
vectorstore.save_local(os.path.join(BASE_PATH, "data", "faiss", "journal_index"))

print("✅ FAISS index created and saved!")

✅ FAISS index created and saved!


In [23]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# Define paths
BASE_PATH = "/Users/Additional Storage/ML Projects/Journal app"
INDEX_PATH = os.path.join(BASE_PATH, "data", "faiss", "journal_index")

# Load embedding model and index
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.load_local(
    INDEX_PATH,
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)
# Run a similarity search
query = "I felt really anxious and overwhelmed this week"
results = vectorstore.similarity_search(query, k=3)  # top 3 relevant entries

# Print results
for i, doc in enumerate(results):
    print(f"\n🔹 Result {i+1}")
    print("📄 Content:", doc.page_content)
    print("🧠 Metadata:", doc.metadata)


🔹 Result 1
📄 Content: The air feels thick this morning, a suffocating weight pressing down on me like a heavy book. My heart pounds in my chest as I glance at the clock—another unpaid bill staring me in the face. The sheets are damp from yesterday’s rain, but my trembling hands still refuse to turn on the heater. 

I’ve been floating between tasks ever since breakfast, my mind racing with worry. Should I finish this report or reach out for help? What if I missed the deadline? My stomach churning as I type frantically, each keystroke a battle against the rising anxiety. 

The kitchen feels empty now, only the faint drip of coffee machines from the other room. I should probably call my parents—I’ve been too busy to think about them—but the thought of their voice on the phone makes my chest tighten. Money is always a wildcard when it comes to my peace of mind. What if they’re worried? What if everything falls apart?

I keep glancing at my watch, each second dragging longer and longer. Sh

In [24]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
def build_rag_prompt(query, retrieved_docs):
    context = "\n\n".join([f"Entry {i+1}:\n{doc.page_content}" for i, doc in enumerate(retrieved_docs)])
    prompt = f"""
You are a therapeutic assistant trained on a user's journal.

Based on the following journal entries:

{context}

Answer the following question with empathy, insight, and practical advice:
{query}
"""
    return prompt
# Load vectorstore
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.load_local(
    INDEX_PATH,
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

# Example query
query = "How do I manage recurring anxiety around work and finances?"
retrieved_docs = vectorstore.similarity_search(query, k=3)

# Build prompt
prompt = build_rag_prompt(query, retrieved_docs)
print(prompt)


You are a therapeutic assistant trained on a user's journal.

Based on the following journal entries:

Entry 1:
The air feels thick this morning, a suffocating weight pressing down on me like a heavy book. My heart pounds in my chest as I glance at the clock—another unpaid bill staring me in the face. The sheets are damp from yesterday’s rain, but my trembling hands still refuse to turn on the heater. 

I’ve been floating between tasks ever since breakfast, my mind racing with worry. Should I finish this report or reach out for help? What if I missed the deadline? My stomach churning as I type frantically, each keystroke a battle against the rising anxiety. 

The kitchen feels empty now, only the faint drip of coffee machines from the other room. I should probably call my parents—I’ve been too busy to think about them—but the thought of their voice on the phone makes my chest tighten. Money is always a wildcard when it comes to my peace of mind. What if they’re worried? What if everyt

In [43]:
import requests
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.chains import LLMChain
from langchain_core.language_models import BaseLLM

import requests
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.chains import LLMChain
#from langchain_core.llms import LLM  # ✅ Use LLM, not BaseLLM
from langchain.llms.base import LLM
OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME = "deepseek-r1"

# ✅ Correct custom wrapper
class DeepSeekLLM(LLM):
    def _call(self, prompt: str, stop=None, run_manager=None) -> str:
        payload = {
            "model": MODEL_NAME,
            "messages": [{"role": "user", "content": prompt}],
            "stream": False
        }
        response = requests.post(OLLAMA_URL, json=payload)
        return response.json()["message"]["content"]

    @property
    def _llm_type(self):
        return "custom-deepseek"

# Use the custom DeepSeek LLM class
llm = DeepSeekLLM()

# Define expected structured response
response_schemas = [
    ResponseSchema(name="query", description="Improved or simplified search query"),
    ResponseSchema(name="filters", description="A dictionary of filters for metadata (e.g., emotions, timestamp, themes)")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Build the prompt
prompt = PromptTemplate(
    template="""You are an intelligent assistant for a journal app.

Given the user query: "{input}",

1. Rewrite it into a search query for journal entries.
2. If the query includes any relevant filters (like emotions, timestamp, or theme), extract them.

{format_instructions}
""",
    input_variables=["input"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

# Combine into a LangChain chain
chain = LLMChain(llm=llm, prompt=prompt)

# Run a test
raw_query = "Can you show my anxious journal entries from last week?"
response = chain.run(input=raw_query)
parsed = output_parser.parse(response)

print("🧠 Structured Query Output:")
print(parsed)

/var/folders/1f/p_pgbb7x59dg0jrjqnvk9pkh0000gn/T/ipykernel_43504/30824729.py:57: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
/var/folders/1f/p_pgbb7x59dg0jrjqnvk9pkh0000gn/T/ipykernel_43504/30824729.py:61: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(input=raw_query)


🧠 Structured Query Output:
{'query': 'Show my journal entries where emotions are anxiety AND timestamp is last week', 'filters': {'emotions': ['anxiety'], 'timestamp': ['last week']}}


In [47]:
# Step 1: Run the LLM chain to extract structured query
parsed = chain.run(input=raw_query)
parsed = output_parser.parse(parsed)

# Step 2: Use structured query from LLM
query = parsed['query']
filters = parsed.get('filters', {})

# Step 3: Run FAISS similarity search (metadata filtering is TODO)
retrieved_docs = vectorstore.similarity_search(query, k=3)

# Step 4: Build the final RAG prompt
prompt = build_rag_prompt(query, retrieved_docs)

# Step 5: Send the prompt to DeepSeek model (your Ollama local call)
payload = {
    "model": MODEL_NAME,
    "messages": [
        {"role": "system", "content": "You are a therapeutic journaling assistant."},
        {"role": "user", "content": prompt}
    ],
    "stream": False
}

print("📩 Final Prompt Sent to DeepSeek:\n")
print(prompt)

response = requests.post(OLLAMA_URL, json=payload)
final_answer = response.json()["message"]["content"]

print("🧠 Answer from DeepSeek:")
print(final_answer)

📩 Final Prompt Sent to DeepSeek:


You are a therapeutic assistant trained on a user's journal.

Based on the following journal entries:

Entry 1:
The room is dimly lit now, the pages of my journal trembling under the weight of the thoughts I’m trying to write. The air conditioning cuts through the silence like a blade, slicing through whatever warmth I managed to hang onto. I sit here, my hands trembling as I flip through the covers, each word a struggle. 

The smell of burning paper hangs heavy in the corners—I’ve been at this long enough, and yet it still feels like yesterday when I started. My mind races, jumping from thought to thought, each one more pressing than the last. I want to write about my failures, but every time I open the book, it’s as if my fingers are slipping away before I can type a single sentence. 

There’s something familiar in this silence—something I’ve heard before, like the sound of broken records trying to play on old vinyl. My heart beats fast, not because

In [62]:
import requests
import pandas as pd
from datetime import datetime
import re
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# ---------- CONFIG ----------
OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME = "deepseek-r1"
CSV_PATH = "chat_log.csv"
ROLLING_WINDOW = 4
INDEX_PATH = "/Users/Additional Storage/ML Projects/Journal app/data/faiss/journal_index"

# ---------- SETUP ----------
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.load_local(INDEX_PATH, embeddings=embedding_model, allow_dangerous_deserialization=True)

chat_history = []
chat_summaries = []
chat_log_rows = []

def strip_think_tags(text):
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()

def summarize_chat_history(history):
    formatted = "\n".join([f"{r.capitalize()}: {c}" for r, c in history])
    prompt = f"""
You are a therapeutic journaling assistant. Do not use first-person language.

Summarize the following conversation in 1–2 emotionally-aware sentences:

{formatted}

Summary:""".strip()

    payload = {
        "model": MODEL_NAME,
        "messages": [{"role": "user", "content": prompt}],
        "stream": False
    }
    try:
        res = requests.post(OLLAMA_URL, json=payload)
        return res.json()["message"]["content"].strip()
    except Exception as e:
        return f"❌ Error: {e}"

def build_rag_prompt(query, retrieved_docs):
    context = "\n\n".join([f"Entry {i+1}:\n{doc.page_content}" for i, doc in enumerate(retrieved_docs)])
    return f"""
You are a therapeutic assistant trained on a user's journal entries.

Based on the following journal entries:

{context}

Answer the following question with empathy, insight, and practical advice:
{query}
""".strip()

def get_deepseek_response_with_rag(chat_memory, user_input):
    # Step 1: Run similarity search
    retrieved_docs = vectorstore.similarity_search(user_input, k=3)

    # Step 2: Build RAG-based context prompt
    rag_prompt = build_rag_prompt(user_input, retrieved_docs)

    # Step 3: Build full message history
    messages = [{
        "role": "system",
        "content": (
            "You are a compassionate therapeutic journaling assistant. "
            "You do NOT roleplay as the user. "
            "Respond with thoughtful, emotionally intelligent advice. "
            "Avoid first-person phrasing. Speak like a guide or support figure."
        )
    }]

    if chat_summaries:
        messages.append({"role": "system", "content": "Previous summary: " + " ".join(chat_summaries)})

    for role, content in chat_memory:
        messages.append({"role": role, "content": content})

    messages.append({"role": "user", "content": rag_prompt})

    payload = {
        "model": MODEL_NAME,
        "messages": messages,
        "stream": False
    }

    try:
        res = requests.post(OLLAMA_URL, json=payload)
        full_response = res.json()["message"]["content"]
        return full_response, rag_prompt, messages, payload
    except Exception as e:
        return f"❌ Error: {e}", rag_prompt, messages, payload

# ---------- MAIN LOOP ----------
while True:
    user_input = input("🧍 You: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    chat_history.append(("user", user_input))
    assistant_reply_raw, rag_prompt_used, messages_used, raw_payload = get_deepseek_response_with_rag(chat_history, user_input)
    assistant_reply_clean = strip_think_tags(assistant_reply_raw)
    chat_history.append(("assistant", assistant_reply_clean))

    print(f"🤖 DeepSeek: {assistant_reply_clean}")

    summary_used = ""
    if len(chat_history) > ROLLING_WINDOW * 2:
        summary_used = summarize_chat_history(chat_history[:-ROLLING_WINDOW*2])
        chat_summaries.append(summary_used)
        chat_history = chat_history[-ROLLING_WINDOW*2:]
        print("🧾 Chat history summarized.")

    row = {
        "timestamp": datetime.now().isoformat(),
        "user_input": user_input,
        "assistant_reply_clean": assistant_reply_clean,
        "assistant_reply_raw": assistant_reply_raw,
        "summary_used": summary_used,
        "last_3_turns": str(chat_history[-6:]),
        "full_context_passed": str(messages_used),
        "rag_prompt_used": rag_prompt_used,
        "raw_prompt": raw_payload["messages"][-1]["content"]
    }

    chat_log_rows.append(row)
    pd.DataFrame(chat_log_rows).to_csv(CSV_PATH, index=False)

🧍 You:  Hey How are you?


🤖 DeepSeek: It seems like you're navigating a deeply personal journey centered on self-acceptance and personal growth. Your entries paint a picture of both contentment with who you are and moments of vulnerability when external pressures or inner struggles arise.

I notice that you've been working on recognizing your worth despite flaws, which is incredibly brave. It's important to acknowledge the effort you're putting in—it shows strength, not weakness.

You also touch on the idea of finding beauty in small acts of self-care, like turning soil or sipping tea. These gestures can offer moments of pause and reflection, allowing you space to rest and reconnect with yourself.

If you ever feel overwhelmed or alone, remember that you don't have to face these feelings alone. Small acts of kindness—like reaching for a cup of tea or connecting with nature—can create spaces for healing and growth.

You're not alone in this journey, and your progress, however small, is enough to celebrate. Keep 

KeyboardInterrupt: Interrupted by user

In [68]:
import requests
import pandas as pd
from datetime import datetime
import re
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# ---------- CONFIG ----------
OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME = "deepseek-r1"
CSV_PATH = "chat_log.csv"
ROLLING_WINDOW = 4
INDEX_PATH = "/Users/Additional Storage/ML Projects/Journal app/data/faiss/journal_index"

# ---------- SETUP ----------
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.load_local(INDEX_PATH, embeddings=embedding_model, allow_dangerous_deserialization=True)

chat_history = []
chat_summaries = []
chat_log_rows = []

def strip_think_tags(text):
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()

def summarize_chat_history(history):
    formatted = "\n".join([f"{r.capitalize()}: {c}" for r, c in history])
    prompt = f"""
You are a therapeutic journaling assistant. Do not use first-person language.

Summarize the following conversation in 1–2 emotionally-aware sentences:

{formatted}

Summary:""".strip()

    payload = {
        "model": MODEL_NAME,
        "messages": [{"role": "user", "content": prompt}],
        "stream": False
    }
    try:
        res = requests.post(OLLAMA_URL, json=payload)
        return res.json()["message"]["content"].strip()
    except Exception as e:
        return f"❌ Error: {e}"

def should_use_rag(user_input):
    prompt = f"""
You are a model that decides whether a user input should use personal journal entries.
Say Yes if the query requires referencing past entries. Say No otherwise.

Examples:
"Hi how are you?" → No  
"What can I do to sleep better?" → No  
"What have I been feeling based on my entries?" → Yes  
"Can you analyze themes from last week’s journal?" → Yes  

Classify:
"{user_input}"
Needs journal context? (Yes/No):"""

    payload = {
        "model": MODEL_NAME,
        "messages": [{"role": "user", "content": prompt}],
        "stream": False
    }
    try:
        res = requests.post(OLLAMA_URL, json=payload)
        return "yes" in res.json()["message"]["content"].lower()
    except:
        return False

def build_rag_prompt(query, retrieved_docs):
    context = "\n\n".join([f"Entry {i+1}:\n{doc.page_content}" for i, doc in enumerate(retrieved_docs)])
    return f"""
You are a therapeutic assistant trained on a user's journal entries.

Based on the following journal entries:

{context}

Answer the following question with empathy, insight, and practical advice:
{query}
""".strip()

def get_deepseek_response(messages):
    payload = {
        "model": MODEL_NAME,
        "messages": messages,
        "stream": False
    }
    res = requests.post(OLLAMA_URL, json=payload)
    return res.json()["message"]["content"]

# ---------- MAIN LOOP ----------
while True:
    user_input = input("🤍 You: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    chat_history.append(("user", user_input))
    use_rag = should_use_rag(user_input)

    messages = [{
        "role": "system",
        "content": (
            "You are a compassionate therapeutic journaling assistant. "
            "You do NOT roleplay as the user. "
            "Respond with thoughtful, emotionally intelligent advice. "
            "Avoid first-person phrasing. Speak like a guide or support figure."
        )
    }]

    if chat_summaries:
        messages.append({"role": "system", "content": "Previous summary: " + " ".join(chat_summaries)})

    for role, content in chat_history:
        messages.append({"role": role, "content": content})

    rag_prompt_used = ""

    if use_rag:
        retrieved_docs = vectorstore.similarity_search(user_input, k=3)
        rag_prompt_used = build_rag_prompt(user_input, retrieved_docs)
        messages.append({"role": "user", "content": rag_prompt_used})
    else:
        messages.append({"role": "user", "content": user_input})
        rag_prompt_used = "RAG not used. Used direct user input."

    assistant_reply_raw = get_deepseek_response(messages)
    assistant_reply_clean = strip_think_tags(assistant_reply_raw)
    chat_history.append(("assistant", assistant_reply_clean))

    print(f"🤖 DeepSeek: {assistant_reply_clean}")

    summary_used = ""
    if len(chat_history) > ROLLING_WINDOW * 2:
        summary_used = summarize_chat_history(chat_history[:-ROLLING_WINDOW*2])
        chat_summaries.append(summary_used)
        chat_history = chat_history[-ROLLING_WINDOW*2:]
        print("📟 Chat history summarized.")

    row = {
        "timestamp": datetime.now().isoformat(),
        "user_input": user_input,
        "assistant_reply_clean": assistant_reply_clean,
        "assistant_reply_raw": assistant_reply_raw,
        "summary_used": summary_used,
        "last_3_turns": str(chat_history[-6:]),
        "full_context_passed": str(messages),
        "rag_prompt_used": rag_prompt_used,
        "raw_prompt": messages[-1]["content"]
    }

    chat_log_rows.append(row)
    pd.DataFrame(chat_log_rows).to_csv(CSV_PATH, index=False)


🤍 You:  Hey, how are you?


🤖 DeepSeek: Hey there! It sounds like you're doing some very important work with these journal entries. Writing about your experiences can be a powerful form of self-care, helping you process emotions, identify patterns, and build resilience. I notice in Entry 1 that you found peace in recognizing your imperfections—a courage worth celebrating.

In Entries 2 and 3, it seems like you're navigating moments of both inner turmoil and finding solace in familiar memories. It’s clear you’re very aware of the weight of expectations and loss, which is a sign of deep self-awareness and strength. The quiet acceptance you’ve adopted feels incredibly brave, especially when facing challenges that bring up turmoil.

I’d like to offer some thoughts on how you might continue processing these emotions:

- **Empathy for Yourself**: It’s okay to feel both the peace from Entry 1 and the turmoil from Entries 2 and 3. These are just two sides of your experience. You don’t have to reconcile them; instead, the

KeyboardInterrupt: Interrupted by user

In [76]:
import requests
import re

OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME = "deepseek-r1"

def strip_think_tags(text):
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()

def needs_journal_context(user_input):
    prompt = f"""
You are an intelligent classifier that decides whether a user's input requires journal-based reflection.

You have access to the user’s past journal entries. Your task is to decide if the question needs referencing or reflecting on past entries (journal context), or if a generic assistant response is enough.

Reply only with "Yes" or "No".

Examples:
---
Input: "What's the weather like today?"
Answer: No

Input: "What themes did I write about this week?"
Answer: Yes

Input: "Can you summarize my last two entries?"
Answer: Yes

Input: "Should I eat more protein?"
Answer: No

Input: "How have I been feeling lately based on my journal?"
Answer: Yes

Now decide for this input:
---
Input: "{user_input}"
Answer:""".strip()

    payload = {
        "model": MODEL_NAME,
        "messages": [{"role": "user", "content": prompt}],
        "stream": False
    }

    try:
        res = requests.post(OLLAMA_URL, json=payload)
        full_reply = res.json()["message"]["content"]
        cleaned_reply = strip_think_tags(full_reply)
        print("🔍 Classifier Response (stripped):\n", cleaned_reply)
        return cleaned_reply.strip().lower().startswith("yes")
    except Exception as e:
        print(f"❌ Error: {e}")
        return False

# Test inputs
test_inputs = [
    "Hey how are you?",
    "What’s the weather like today?",
    "What did I journal about last week?",
    "Can you summarize my past three entries?",
    "What did I say about burnout?",
    "Should I eat more protein?",
    "Tell me how I’ve been feeling lately",
    "What are recurring patterns in my journal?",
    "What did I say about my breakup last month?",
    "Remind me what made me anxious recently",
    "What did I write about therapy last week?",
    "Where did I feel most at peace in my past entries?",
    "How do I usually feel on Mondays?",
    "What’s something positive I wrote this week?"
]

# Run tests
for query in test_inputs:
    print(f"\n🧍 Input: {query}")
    needs_journal_context(query)


🧍 Input: Hey how are you?
🔍 Classifier Response (stripped):
 Answer: No

🧍 Input: What’s the weather like today?
🔍 Classifier Response (stripped):
 No

🧍 Input: What did I journal about last week?
🔍 Classifier Response (stripped):
 Yes

🧍 Input: Can you summarize my past three entries?
🔍 Classifier Response (stripped):
 Yes

🧍 Input: What did I say about burnout?
🔍 Classifier Response (stripped):
 Yes

🧍 Input: Should I eat more protein?
🔍 Classifier Response (stripped):
 Answer: No

🧍 Input: Tell me how I’ve been feeling lately
🔍 Classifier Response (stripped):
 Answer: No

🧍 Input: What are recurring patterns in my journal?
🔍 Classifier Response (stripped):
 Yes

🧍 Input: What did I say about my breakup last month?
🔍 Classifier Response (stripped):
 Yes

🧍 Input: Remind me what made me anxious recently
🔍 Classifier Response (stripped):
 Yes

🧍 Input: What did I write about therapy last week?
🔍 Classifier Response (stripped):
 Yes

🧍 Input: Where did I feel most at peace in my past e

In [78]:
test_cases = [
    # (Input, Expected Output)
    ("What did I write about burnout last week?", "Yes"),
    ("Can you tell me how I've been feeling lately?", "Yes"),
    ("What's the weather like tomorrow?", "No"),
    ("Summarize my last three journal entries.", "Yes"),
    ("Define imposter syndrome.", "No"),
    ("How do I usually cope with stress based on my entries?", "Yes"),
    ("Play lo-fi music for focus.", "No"),
    ("What are some recurring themes in my journal?", "Yes"),
    ("What are signs of depression?", "No"),
    ("Tell me how I’ve changed over the past month.", "Yes"),
    ("How can I stop procrastinating?", "No"),
    ("What emotions have I been expressing recently?", "Yes"),
    ("What are some high-protein vegan meals?", "No"),
    ("Did I mention my mom in any recent entries?", "Yes"),
    ("Give me a motivational quote.", "No"),
    ("Do I often write about loneliness?", "Yes")
]

# Run tests and show predictions
for query, expected in test_cases:
    print(f"\n🧍 Input: {query}")
    prediction = needs_journal_context(query)
    print(f"✅ Expected: {expected} | 🔍 Predicted: {'Yes' if prediction else 'No'}")


🧍 Input: What did I write about burnout last week?
🔍 Classifier Response (stripped):
 Yes
✅ Expected: Yes | 🔍 Predicted: Yes

🧍 Input: Can you tell me how I've been feeling lately?
🔍 Classifier Response (stripped):
 Yes
✅ Expected: Yes | 🔍 Predicted: Yes

🧍 Input: What's the weather like tomorrow?
🔍 Classifier Response (stripped):
 No
✅ Expected: No | 🔍 Predicted: No

🧍 Input: Summarize my last three journal entries.
🔍 Classifier Response (stripped):
 Answer: Yes
✅ Expected: Yes | 🔍 Predicted: No

🧍 Input: Define imposter syndrome.
🔍 Classifier Response (stripped):
 Answer: No
✅ Expected: No | 🔍 Predicted: No

🧍 Input: How do I usually cope with stress based on my entries?
🔍 Classifier Response (stripped):
 Yes
✅ Expected: Yes | 🔍 Predicted: Yes

🧍 Input: Play lo-fi music for focus.
🔍 Classifier Response (stripped):
 Answer: No
✅ Expected: No | 🔍 Predicted: No

🧍 Input: What are some recurring themes in my journal?
🔍 Classifier Response (stripped):
 Yes
✅ Expected: Yes | 🔍 Predicted: 

In [96]:
import requests
import pandas as pd
from datetime import datetime
import re
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# ---------- CONFIG ----------
OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME = "deepseek-r1"
CSV_PATH = "chat_log.csv"
ROLLING_WINDOW = 4
INDEX_PATH = "/Users/Additional Storage/ML Projects/Journal app/data/faiss/journal_index"

# ---------- SETUP ----------
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.load_local(INDEX_PATH, embeddings=embedding_model, allow_dangerous_deserialization=True)

chat_history = []
chat_summaries = []
chat_log_rows = []

def strip_think_tags(text):
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()

def summarize_chat_history(history):
    formatted = "\n".join([f"{r.capitalize()}: {c}" for r, c in history])
    prompt = f"""
You are a therapeutic journaling assistant. Do not use first-person language.

Summarize the following conversation in 1–2 emotionally-aware sentences:

{formatted}

Summary:""".strip()

    payload = {
        "model": MODEL_NAME,
        "messages": [{"role": "user", "content": prompt}],
        "stream": False
    }
    try:
        res = requests.post(OLLAMA_URL, json=payload)
        return res.json()["message"]["content"].strip()
    except Exception as e:
        return f"❌ Error: {e}"

def should_use_rag(user_input):
    prompt = f"""
You are an intelligent classifier that decides whether a user's input requires journal-based reflection.

You have access to the user’s past journal entries. Your task is to decide if the question needs referencing or reflecting on past entries (journal context), or if a generic assistant response is enough.

Reply only with "Yes" or "No".

Examples:
---
Input: "What's the weather like today?"
Answer: No

Input: "Should I eat more protein?"
Answer: No

Input: "What did I write about this week?"
Answer: Yes

Input: "Can you summarize my past entries?"
Answer: Yes

Input: "How do I usually feel on Mondays?"
Answer: Yes

Input: "Can you not find out based on my journal?"
Answer: Yes

Input: "What have you noticed from my writing?"
Answer: Yes

Input: "Just tell me what you think I've been going through lately."
Answer: Yes

Input: "Hi, how are you?"
Answer: No

Now decide for this input:
---
Input: "{user_input}"
Answer:""".strip()

    payload = {
        "model": MODEL_NAME,
        "messages": [{"role": "user", "content": prompt}],
        "stream": False
    }
    try:
        res = requests.post(OLLAMA_URL, json=payload)
        full_reply = res.json()["message"]["content"]
        cleaned_reply = strip_think_tags(full_reply)
        return cleaned_reply.strip().lower().startswith("yes")
    except Exception as e:
        print(f"❌ RAG classifier error: {e}")
        return False

def build_rag_prompt(query, retrieved_docs):
    context = "\n\n".join([f"Entry {i+1}:\n{doc.page_content}" for i, doc in enumerate(retrieved_docs)])
    return f"""
You are a therapeutic assistant trained on a user's journal entries.

Based on the following journal entries:

{context}

Answer the following question with empathy, insight, and practical advice:
{query}
""".strip()

def get_deepseek_response(messages):
    payload = {
        "model": MODEL_NAME,
        "messages": messages,
        "stream": False
    }
    res = requests.post(OLLAMA_URL, json=payload)
    return res.json()["message"]["content"]

# ---------- MAIN LOOP ----------
while True:
    user_input = input("🧽 You: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    chat_history.append(("user", user_input))
    use_rag = should_use_rag(user_input)
    print("📊 RAG needed?", use_rag)

    messages = [{
        "role": "system",
        "content": (
            "You are a compassionate therapeutic journaling assistant. "
            "You do NOT roleplay as the user. "
            "Respond with thoughtful, emotionally intelligent advice. "
            "Avoid first-person phrasing. Speak like a guide or support figure."
        )
    }]

    if chat_summaries:
        messages.append({"role": "system", "content": "Previous summary: " + " ".join(chat_summaries)})

    for role, content in chat_history:
        messages.append({"role": role, "content": content})

    rag_prompt_used = ""

    if use_rag:
        retrieved_docs = vectorstore.similarity_search(user_input, k=3)
        rag_prompt_used = build_rag_prompt(user_input, retrieved_docs)
        messages.append({"role": "user", "content": rag_prompt_used})
    else:
        messages.append({"role": "user", "content": user_input})
        rag_prompt_used = "RAG not used. Used direct user input."

    assistant_reply_raw = get_deepseek_response(messages)
    assistant_reply_clean = strip_think_tags(assistant_reply_raw)
    chat_history.append(("assistant", assistant_reply_clean))

    print(f"🤖 DeepSeek: {assistant_reply_clean}")

    summary_used = ""
    if len(chat_history) > ROLLING_WINDOW * 2:
        summary_used = summarize_chat_history(chat_history[:-ROLLING_WINDOW*2])
        chat_summaries.append(summary_used)
        chat_history = chat_history[-ROLLING_WINDOW*2:]
        print("📿 Chat history summarized.")

    row = {
        "timestamp": datetime.now().isoformat(),
        "user_input": user_input,
        "assistant_reply_clean": assistant_reply_clean,
        "assistant_reply_raw": assistant_reply_raw,
        "summary_used": summary_used,
        "last_3_turns": str(chat_history[-6:]),
        "full_context_passed": str(messages),
        "rag_prompt_used": rag_prompt_used,
        "raw_prompt": messages[-1]["content"]
    }

    chat_log_rows.append(row)
    pd.DataFrame(chat_log_rows).to_csv(CSV_PATH, index=False)

🧽 You:  Hey how are you doing


📊 RAG needed? False
🤖 DeepSeek: The phrase "Hey how are you doing" serves as a casual and friendly way to inquire about someone's well-being. Breaking it down:

1. **Greeting ("Hey")**: This acts as an initial call to action, typically used in informal settings.

2. **Question Component ("how are you doing")**: This part asks about the speaker's health or general state of being, often with a positive tone.

Cultural nuances may affect how this phrase is perceived, with some cultures valuing indirect communication over direct ones. The cadence and tone can also influence its interpretation, though generally, it remains a polite way to ask about someone's well-being without implying sarcasm unless context suggests otherwise.

In summary, "Hey how are you doing" is primarily a friendly and neutral way to check in on someone's health or mood, often used informally across various cultural settings.


KeyboardInterrupt: Interrupted by user

📊 RAG needed? True
